# FIRST: Go to Runtime -> Change Runtime type -> GPU accelerated


This next codeblock installs an archived version of transformers

installs datasets==1.16.1 for putting example datasets (might not be needed)

bitsandbytes for tokenizing

In [1]:
!pip uninstall -y transformers && pip install --no-cache-dir https://github.com/deniskamazur/transformers/archive/gpt-j-8bit.zip
!pip install datasets==1.16.1 # for loading example datasets (might not be needed)
!pip install bitsandbytes # for tokenizing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     \ 10.2 MB 647 kB/s
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 880 kB 8.7 MB/s 
     |████████████████████████████████| 7.6 MB 74.8 MB/s 
     |████████████████████████████████| 163 kB 57.2 MB/s 
  Created wheel for transformers: filename=transformers-4.17.0.dev0-py3-none-any.whl size=3653108 sha256=eb0d2406af3c628d776260faf851d454da593663c3acf096471d1eea00c61d11
  Stored in directory: /tmp/pip-ephem-wheel-cache-oz0imjx_/wheels/f7/18/de/b62fe0bb6c170d5f315f8673652d2bfc776b847b3fd26553fb
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=f13395713509b0de2f4785c244d4aba9d24b3df6f488a09e68ea4ce894bcb518
  Stored in directory: /tmp/pip-ephem-wheel-cache-oz0imjx_/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079ba

# Below loads model

In [2]:
def gpu_model():
  import torch
  import transformers
  from transformers.models.gptj import GPTJForCausalLM #gptj only?
  device = 'cuda' if torch.cuda.is_available() else 'cpu' # select gpu otherwise use cpu
  tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B") #loads original 6B model
  model = GPTJForCausalLM.from_pretrained("hivemind/gpt-j-6B-8bit", low_cpu_mem_usage=True).to(device) #use pretrained 8bit "shrunken" model used for low memory aka using for google colab
  return [device,tokenizer,model]
  
def cpu_model():
  import torch
  import transformers
  from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
  device = 'cuda' if torch.cuda.is_available() else 'cpu' # select gpu otherwise use cpu
  tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
  model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")
  return [device,tokenizer,model]



# Install youtubetranscript api for text

In [3]:
!pip install youtube_transcript_api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Summary pipeline:

In [4]:
!pip install fastapi nest-asyncio pyngrok uvicorn
!ngrok authtoken 2B39YnQIGeHG0FeFnLIgkPVJmdq_3eXfqZe6aXCu8YkFZ2nQ9

# !pip install flask
# !pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.7 MB/s 
     |████████████████████████████████| 745 kB 19.2 MB/s 
     |████████████████████████████████| 56 kB 5.4 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 80 kB 10.0 MB/s 
     |████████████████████████████████| 58 kB 6.5 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=5126dda98fa0ce6754360717ba8ce488b38e5be406e6ed8fb1ba0bad083da723
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [5]:
!ngrok http -host-header=rewrite localhost:8000

Traceback (most recent call last):
  File "/usr/lib/python3.7/subprocess.py", line 1019, in wait
    return self._wait(timeout=timeout)
  File "/usr/lib/python3.7/subprocess.py", line 1653, in _wait
    (pid, sts) = self._try_wait(0)
  File "/usr/lib/python3.7/subprocess.py", line 1611, in _try_wait
    (pid, sts) = os.waitpid(self.pid, wait_flags)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/ngrok", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/pyngrok/ngrok.py", line 501, in main
    run(sys.argv[1:])
  File "/usr/local/lib/python3.7/dist-packages/pyngrok/ngrok.py", line 489, in run
    process.run_process(pyngrok_config.ngrok_path, args)
  File "/usr/local/lib/python3.7/dist-packages/pyngrok/process.py", line 339, in run_process
    subprocess.call(start)
  File "/usr/lib/python3.7/subprocess.py", line 341, in call
    return p.wait(timeout=t

In [6]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
from fastapi import FastAPI, Request
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from youtube_transcript_api import YouTubeTranscriptApi
import torch
from typing import Union
from pydantic import BaseModel
import re
from transformers import pipeline
from typing import TypeVar
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
import json


# load models and store in variables
dev_tok_model = cpu_model()
tokenizer = dev_tok_model[1]
model = dev_tok_model[2]
# 

T = TypeVar("T")


class Result:
    def __init__(self, ok: bool, value: T = None, error: str = ""):
        self.ok = ok
        # If the result is ok, set the value. Otherwise, set the error
        self.value = value if ok else None
        self.error = error if not ok else ""

    def __str__(self):
        # If the result is ok, return the value. Otherwise, return the error
        return f"Ok: {self.value}" if self.ok else f"Err: {self.error}"

    def __repr__(self):
        return str(self)

    def is_ok(self):
        return self.ok

    def is_err(self):
        return not self.ok

    def unwrap(self):
        if self.is_ok():
            return self.value
        else:
            raise Exception(self.error)

    def match(self, ok_func, err_func):
        if self.is_ok():
            return ok_func(self.value)
        else:
            return err_func(self.error)

    # func must return a Result
    def flat_map(self, func):
        if self.is_ok():
            return func(self.value)
        else:
            return self


is_youtube_regex = re.compile(r"https?://(www\.)?youtube\.com")


def is_youtube(link: str) -> bool:
    """Check if link is a YouTube link"""
    return is_youtube_regex.match(link) is not None


youtube_video_id_regex = re.compile(r"https?://(?:(?:(?:www\.youtube\.com|m\.youtube\.com)/watch\?v=([0-9A-Za-z_-]{10}[048AEIMQUYcgkosw]).*)|(?:youtu\.be/([0-9A-Za-z_-]{10}[048AEIMQUYcgkosw])))")


def get_youtube_video_id(link: str) -> Result:
    """Get the video id from a YouTube link"""
    if not is_youtube(link):
        return Result(False, error="Not a youtube link")
    match = youtube_video_id_regex.match(link)
    if match is None:
        return Result(False, error="Invalid youtube link")
    return Result(True, value=match.group(1))


def get_transcript(video_id: str) -> Result:
    try:
        text = YouTubeTranscriptApi.get_transcript(video_id)
        # We need to transform the transcript into a string
        return Result(True, value=" ".join([line["text"] for line in text]))
    except Exception as e:
        return Result(False, error=str(e))


def get_summary(text: str) -> Result:
    # Gen must be a transformers pipeline
    try:
        prompt = text + "\n TLDR: "
        # WARNING: This is designed around gpt-neo-125M, which can be called in the following way
        # Other models may require different parameters, and different ways of obtaining the summary
        # Read through the documentation of the models for proper usage
        # summary = gen(prompt, do_sample=True, temperature=0.9, max_new_tokens=200)[0]["generated_text"]

        """needs change"""
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids #.cuda() # .cuda() is for gpu, remove if not needed, gpt-j-6b needs gpu.

        gen_tokens = model.generate(

            input_ids,

            do_sample=True,

            temperature=0.9,

            max_length=2000,

        )


        summary = tokenizer.batch_decode(gen_tokens)[0]
        """needs change"""

        return Result(True, value=summary)
    except Exception as e:
        return Result(False, error=str(e))


def link_to_summary(link: str) -> Result:
    return get_youtube_video_id(link) \
        .flat_map(get_transcript) \
        .flat_map(lambda text: get_summary(text))


def link_to_transcript(link: str) -> Result:
    return get_youtube_video_id(link) \
        .flat_map(get_transcript)


def link_to_prompt(link: str) -> Result:
    return get_youtube_video_id(link) \
        .flat_map(get_transcript) \
        .flat_map(lambda text: Result(True, value=text + "\n TLDR: "))


def unwrap_result(result: Result):
    if result.is_ok():
        return result.value
    else:
        raise Exception(result.error)


def main(link):
    #['audio-classification', 'automatic-speech-recognition', 'conversational', 'feature-extraction', 'fill-mask', 'image-classification', 'image-segmentation', 'ner', 'object-detection', 'question-answering', 
    #'sentiment-analysis', 'summarization', 'table-question-answering', 'text-classification', 'text-generation', 'text2text-generation', 'token-classification', 'translation', 'zero-shot-classification', 'translation_XX_to_YY']
    # pipe = pipeline('text-generation', model='EleutherAI/gpt-neo-125M') # need to remove this line?
    result = link_to_summary(link)
    return result

# load server app
app = FastAPI()
# enable cors
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get('/index')
async def home():
  result = main('https://www.youtube.com/watch?v=P7yM0TKvUm4')
  return result
"""
class Item(BaseModel):
    id: Union[int, None] = None
    link: str
    summary: Union[str, None] = None
    rating: Union[int, None] = None

@app.post("/items")
async def getInformation(item : Request):
    req_info = await item.json()
    # persist id
    # delete link
    # retreive summary
    # keep same rating.
    print(type(req_info))
    # link = req_info['link'] #put back
    # summary = main(link) #put back
    req_info['summary'] = "summary" #put back
    req_info['link'] = ""
    
    return {
        "status" : "SUCCESS",
        "data" : req_info,
        "type" : str(type(req_info))
    }
 """


@app.post("/items")
async def getInformation(item : Request):
    req_info = await item.json()
    # persist id
    # delete link
    # retreive summary
    # keep same rating.
    print(type(req_info))
    link = req_info['link']
    summary = main(link)
    req_info['summary'] = summary 
    req_info['link'] = ""
    
    return {
        "status" : "SUCCESS",
        "data" : req_info,
        "type" : str(type(req_info))
    }

# @app.post("/items/")
# async def create_item(item: Item):
#     return item

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://845e-35-247-3-235.ngrok.io


INFO:     Started server process [136]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2607:fb90:8ab6:c752:ccf8:897a:1df8:8470:0 - "OPTIONS /items HTTP/1.1" 200 OK
<class 'dict'>
INFO:     2607:fb90:8ab6:c752:ccf8:897a:1df8:8470:0 - "POST /items HTTP/1.1" 200 OK
<class 'dict'>
INFO:     2607:fb90:8ab6:c752:ccf8:897a:1df8:8470:0 - "POST /items HTTP/1.1" 200 OK
<class 'dict'>
INFO:     172.58.180.237:0 - "POST /items HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [136]


In [8]:
from youtube_transcript_api import YouTubeTranscriptApi

raw_data = YouTubeTranscriptApi.get_transcript("P7yM0TKvUm4")

# print(raw_data)
type(raw_data)
# first part of transcript data
print(raw_data[0]["text"])
text = ""
lis_text = []
for i, e in enumerate(raw_data):
  lis_text.append(raw_data[i]["text"])
# list to string
text = ''.join(lis_text)
print(text)



prompt = text + "\n TLDR:"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids #.cuda() # .cuda() is for gpu, remove if not needed, gpt-j-6b needs gpu.

gen_tokens = model.generate(

    input_ids,

    do_sample=True,

    temperature=0.9,

    max_length=2000,

)


gen_text = tokenizer.batch_decode(gen_tokens)[0]

In the wealthiest circles 
of Victorian England,
In the wealthiest circles 
of Victorian England,bizarre fads ran rampant.But perhaps none was as strange
as the tapeworm diet,in which dieters swallowed
an unhatched tapewormand let it grow inside them 
by consuming undigested meals.Obviously, this is an exceptionally
dangerous and unhealthy wayto manage your weight.However, while modern fad diets
aren't usually this extreme,they do promise similar results;
specifically, losing weight fast.So, are there any fast diets that do work?And are any of them 
actually healthy for you?To answer these questions, 
let’s consider a thought experiment.Sam and Felix are identical twins
both planning to go on a diet.They share the same height, weight,
fat and muscle mass.But Sam is hoping to lose weight slowly,
while Felix wants to go fast.Sam's plan is to gradually decrease
his calorie intakeand increase his regular exercise.With less energy coming in
and more being expended,he’s creating an energy de

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-1bab6eb46b9d>", line 31, in <module>
    max_length=2000,
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py", line 1103, in generate
    inputs_tensor, model_kwargs, model_input_name
  File "/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py", line 510, in _prepare_encoder_decoder_kwargs_for_generation
    model_kwargs["encoder_outputs"]: ModelOutput = encoder(**encoder_kwargs)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/transformers/models/

KeyboardInterrupt: ignored

FROM THE ABOVE: we print the type and len of input_ids because unsure what it does

FROM THE ABOVE: .cuda() function uses our gpu, (if you delete it, code might still work?)

BELOW: print gen_text

In [ ]:
print(type(input_ids))
print(len(input_ids))
print(gen_text)

In [ ]:
import tensorflow
tensorflow.test.is_gpu_available()

# Things that need doing...?

For App:
- Ability to make API call to this model
- Ability to append Summaries to database

For Experiments:
- Code that lets us loop through our summaries
- Metrics for different models? Many metrics for same model? Different prompts?